# Manipulating Experiments

The MouseConnectivityCache AllenSDK class provides methods for downloading metadata about experiments, including their viral injection site and the mouse's transgenic line.  You can request information either as a Pandas DataFrame or a simple list of dictionaries.  This demonstrates the former.

An important feature of the MouseConnectivityCache is how it stores and retrieves data for you.  By default, it will create (or read) a manifest file that keeps track of where various connectivity atlas data are stored.  If you request something that has not already been downloaded, it will download it and store it in a well known location.  

The hard drives provided to you have a connectivity atlas manifest that indexes a the majority of the connectivity atlas data, including gridded projection volumes at 25um.  The script that downloaded all of this data is also provided.

In [ ]:
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache

# the manifest file is a simple JSON file that keeps track of all of
# the data that has already been downloaded onto the hard drives 
# we have provided for you.
mcc = MouseConnectivityCache(manifest_file='manifest.json')

# open up a list of all of the experiments
all_experiments = mcc.get_experiments(dataframe=True)
print "%d total experiments" % len(all_experiments)

# take a look at what we know about an experiment with a primary motor injection
all_experiments.loc[122642490]

MouseConnectivityCache has a method for retrieving the adult mouse structure ontology as an Ontology class instance.  This is a light wrapper around a Pandas DataFrame of structures that provides some helper methods for figuring out which structures are parents/children of each other.  It's also handy for finding the ID of a structure by its acronym.

In [ ]:
# grab the Ontology instance
ontology = mcc.get_ontology()

# get some info on the isocortex
isocortex = ontology.get_structure_by_acronym('Isocortex')
isocortex

In [ ]:
# fetch the experiments that have injections in the isocortex of cre-positive mice
cre_cortical_experiments = mcc.get_experiments(cre=True, 
                                               injection_structure_ids=isocortex['id'])

print "%d cre cortical experiments" % len(cre_cortical_experiments)

print [ e['transgenic-line'] for e in cre_cortical_experiments ][:5]

# Signal Unionization

The ProjectionStructureUnionizes API data tells you how much signal there was in a given structure and experiment.  It contains the density of projecting signal, volume of projecting signal, and other information.  MouseConnectivityCache provides methods for querying and storing this data.

In [ ]:
visp = ontology.get_structure_by_acronym('VISp')
visp_experiments = mcc.get_experiments(cre=False, 
                                       injection_structure_ids=visp['id'])

print "%d VISp experiments" % len(visp_experiments)

structure_unionizes = mcc.get_structure_unionizes([ e['id'] for e in visp_experiments ], 
                                                  is_injection=False,
                                                  structure_ids=isocortex['id'])

print "%d VISp non-injection, cortical structure unionizes" % len(structure_unionizes)

In [ ]:
structure_unionizes

In [ ]:
dense_unionizes = structure_unionizes[ structure_unionizes.projection_density > .5 ]
large_unionizes = dense_unionizes[ dense_unionizes.volume > .5 ]
large_structures = ontology[set(large_unionizes.structure_id)]

print "%d large, dense, cortical, non-injection unionizes, %d structures" % ( len(large_unionizes), len(large_structures) )

print large_structures.name

large_unionizes

# Manipulating Grid Data

The MouseConnectivity class also helps you download and open every experiment's projection grid data volume.  For this course, we've provided you with the 25um volumes.  You could also download data at other resolutions if you prefer (10um, 50um, 100um).

This demonstrates how you can load the projection density for a particular experiment.  It also shows how to download the template volume to which all grid data is registered.  Voxels in that template have been structurally annotated by neuroanatomists and stored in a separate annotation volume image.

In [ ]:
pd, pd_info = mcc.get_projection_density(122642490)
template, template_info = mcc.get_template_volume()
annot, annot_info = mcc.get_annotation_volume()

print pd_info
print pd.shape, template.shape, annot.shape

Once you have these loaded, you can use matplotlib see what they look like.  

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# pick a slice to show
slice_idx = 264

# show that slice of all three volumes side-by-side
f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(15,3))
ax1.imshow(pd[slice_idx,:,:], cmap='hot', aspect='equal')
ax2.imshow(template[slice_idx,:,:], cmap='gray', aspect='equal', vmin=template.min(), vmax=template.max())
ax3.imshow(annot[slice_idx,:,:], cmap='gray', aspect='equal', vmin=0, vmax=2000)

The MouseConnectivityCache also provides you with a way to compute and store structure masks.  A structure mask is a numpy array that tells you if a voxel belongs to a particular structure.  

How is this different from the annotation volume?  The annotation volume assigns each voxel a single label, usually one that is at the bottom of the structure ontology.  You won't find a single voxel labeled 'Isocortex' in the annotation volume, but there are many voxels that are labeled as descendants of the Isocortex.  MouseConnectivityCache can compute for you a mask of voxels that are labeled as any of the descendants of a structure.

Computing masks is a time-consuming task, so we precomputed them for you.

In [ ]:
isocortex_mask, _ = mcc.get_structure_mask(315)

# pull out the values of all voxels in the isocortex mask
isocortex_pd = pd[isocortex_mask > 0]

# print out the average projection density of voxels in the isocortex
print isocortex_pd.mean()

# Excercises

1) Correlation Search: take an experiment you find interesting (search http://connectivity.brain-map.org) and compute for yourself which experiments have the most correlated projection density patterns within a particular structure (say, isocortex).

2) Unionization: try to reproduce the values you see in the structure unionizes table for one experiment.  Hint: the fastest way to do this is to iterate over the annotation volume voxels.  The structure masks will slow you down!

3) Structure Centroids: compute the centroid of the left hemisphere of the isocortex.